In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase

db_user="root"
db_password=""
db_host="localhost"
db_port=3306
db_name="langchain_db"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")


In [8]:
print(db.table_info)
print(db.dialect)
print(db.get_usable_table_names())


CREATE TABLE table_1 (
	id INTEGER(11) NOT NULL, 
	name VARCHAR(300) NOT NULL, 
	age INTEGER(11) NOT NULL, 
	salary INTEGER(11) NOT NULL, 
	PRIMARY KEY (id)
)ENGINE=InnoDB COLLATE utf8mb4_general_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from table_1 table:
id	name	age	salary
1	Zubayer Isfar	23	100000
2	Tariq Zabir	20	50000
3	Mahir 	22	70000
*/
mysql
['table_1']


In [9]:
from langchain.chains import create_sql_query_chain
from langchain_ollama import ChatOllama

model=ChatOllama(
    model="gemma3:1b",
    temperature=0.4,
)


query_generate=create_sql_query_chain(model,db)
query_generate

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables

In [16]:
query_generator=query_generate.invoke({
    "question": "Which age are more than 20?"
})
query_generator

'```sql\nSELECT age FROM table_1 WHERE age > 20\n```'

In [ ]:
from langchain_community.tools import QuerySQLDataBaseTool

query_executor = QuerySQLDataBaseTool(db=db)

full_chain = query_generate | query_executor

full_chain.invoke({
    "question": "What are the names of those who are more than 20 in age?"
})

"[('Zubayer Isfar',), ('Mahir ',)]"